### 1. load in your student data
### 2. left join student data with the kcs you've identified for the problem that the row is working on



## Running Gemini Extraction



In [ ]:
!pip install --upgrade google-generativeai
import google.generativeai as genai
from tqdm import tqdm
import os

In [ ]:
GEMINI_API_KEY = [TODO ENTER API KEY HERE]
genai.configure(api_key=GEMINI_API_KEY)

#choose your model (flash 2.0 is fast)
model = genai.GenerativeModel("gemini-2.0-flash")

### In case extraction does not run through (e.g your notebook stops prematurely after 20 minutes), save all the outputs as txt files in a directory so if the txt exists it does not need to be regenerated.


In [ ]:
result_directory = "/content/drive/MyDrive/gemini_labels_KClist"
os.makedirs(result_directory, exist_ok=True)

### Feel Free to Edit Prompt (here is example)

In [ ]:
def construct_prompt(student_code, solution, consensus_str):
    concepts_str = consensus_str
    prompt = f"""

    You are a helpful concept classifier.
    You will be given:
    - A student's code submission
    - The official solution code
    - A list of concepts

    Your task:
    For each concept code in the list, determine a label:
     1 if the concept is present and correctly used by the student's code
     -1 of the concept is misunderstood (incorrect usage) in the student’s current code or if the concept is missing but required to complete a valid solution for the problem
     0 if the concept is not present in the student’s current code and is not required to complete a valid solution for the problem

    For each concept in the concepts_str, output a line in the form:
    concept_code:label (for example, 1.1.4.1 Statements & Expressions: 1)

    Do not include any explanations, just the mappings.

    # Student's Code:
    {student_code}

    # Solution:
    {solution}

    # Concepts:
    {concepts_str}
    """

    return prompt.strip()

In [ ]:
def get_gemini_labels(prompt, i):
    try:
      if i % 2 == 0:

        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Error during gemini call: {e}")
        return "error"


Make Sure to Fill in TODOs below

In [ ]:
from time import sleep
result_directory = "/content/drive/MyDrive/61abot/gemini_labels_KClist"

os.makedirs(result_directory, exist_ok=True)
for i, row in tqdm(df.iterrows(), total=len(df)):

    print(i)

    if os.path.exists(os.path.join(result_directory, f"{i}.txt")):
        continue
    #ToDO (enter name column title for the students code)
    student_code = row['extractedFunction_validonlyforpython']
    #ToDO (enter name column title for the solution)
    solution = row['solution']
    #ToDO (enter name column title for the KC List associated with problem)
    consensus_str = row['consensus_kc_labeling']
    prompt = construct_prompt(student_code, solution, consensus_str)

    try:
      sleep(5)
      response = get_gemini_labels(prompt)
    except Exception as e:
      print(e)
      continue


    with open(os.path.join(result_directory, f"{i}.txt"), "w") as f:
        f.write(response)

  #once all txt files show, you can create whatever format you prefer to store data